## Korrelation

In [75]:
#Imports

import pandas as pd
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import numpy as np
import modules

Spalten laden:

In [76]:
col_list=['employer_num_employees' , "wage_offer_from_9089", "wage_offered_from_9089"]
visas_df = pd.read_csv("data/us_perm_visas.csv" , usecols=col_list)

C:\Users\alina\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (148) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Wage offer clean merge:

In [77]:
def clean_wage_offer_from(inital_df = pd.DataFrame):
    col_list = ["wage_offer_from_9089", "wage_offered_from_9089"]
    temp_df = inital_df[col_list]

    #Firstly, all string types will be transformed into a format, which can be converted into a float type.
    #Secondly every cell is converted into float.
    temp_df["wage_offer_from_9089"] = temp_df["wage_offer_from_9089"].apply(clean_currency).astype('float')

    #The two different columns 'wage_offer_from_9089' and 'wage_offered_from_9089' get merged because they both contain similar information and don't overap each other.
    #The columns get merged through simmple addition, therefore all na values are replaced by zero.
    temp_df['wage_offer_from_merged'] = temp_df['wage_offer_from_9089'].fillna(0) + temp_df['wage_offered_from_9089'].fillna(0)

    #Cells which contain zero, even after the merge, will get converted into NaN values. Providing better data for the subsequent analysis.
    temp_df['wage_offer_from_merged'].replace(0, np.nan, inplace=True)

    #only the cleaned up column 'wage_offer_merged' gets returned
    return temp_df['wage_offer_from_merged']


In [78]:
def clean_currency(x):
    """ If the value is a string, then remove delimiters
    otherwise, the value is numeric and can be converted.
    Additionally the provided data contains two '#############' values. These will get replaced by NaN.
    Source: https://pbpython.com/currency-cleanup.html (with slight changes)
    """
    if isinstance(x, str):
        if x == '#############':
            x = np.nan
        else:
            return(x.replace(',', ''))
    return(x)

In [89]:
#Aus NaN soll null werden, geht noch nicht
visas_df['employer_num_employees'] = visas_df['employer_num_employees'].replace(r'^\s*$', np.nan, regex=True)
visas_df['employer_num_employees'] = visas_df['employer_num_employees'].replace('NaN', '0')

In [88]:
visas_df['wage_clean'] = clean_wage_offer_from(visas_df)
visas_df.sample(20)

C:\Users\alina\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\alina\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\alina\anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

,employer_num_employees,wage_offer_from_9089,wage_offered_from_9089,wage_clean
364115,31000.0,"92,144.00",NaN,92144.00
118998,NaN,NaN,92539.0,92539.00
127666,NaN,NaN,55946.8,55946.80
355891,800.0,"187,000.00",NaN,187000.00
52516,NaN,NaN,43.0,43.00
169895,110000.0,"78,832.00",NaN,78832.00
227563,15000.0,69327,NaN,69327.00
112144,NaN,NaN,56493.0,56493.00
52756,NaN,NaN,160000.0,160000.00
58077,NaN,NaN,125962.0,125962.00


Korrelations-Tests:

Regression

In [81]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [82]:
x = np.array(visas_df['wage_clean']).reshape((-1, 1))
y = np.array(visas_df['employer_num_employees'])

In [83]:
print(x)
print(y)

[[75629.]
 [37024.]
 [47923.]
 ...
 [79082.]
 [37981.]
 [84926.]]
[    nan     nan     nan ... 3.3e+01 1.6e+01 5.4e+04]


In [90]:
model = LinearRegression()
model.fit(x, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
>>> r_sq = model.score(x, y)
>>> print('coefficient of determination:', r_sq)